In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from classificatiom_model import EarlyStopping, ModelSaver,train_classification_model,plot_learning_curves
from classificatiom_model import evaluate_classification_model as evaluate_model

In [ ]:
import torchvision.models as models  # 导入PyTorch视觉模型库
# 导入必要的库
import torch.nn as nn  # 导入神经网络模块
import torchvision.models as models  # 重复导入，可以删除
from torchsummary import torchsummary  # 导入模型可视化工具，用于显示模型参数统计
from torchviz import make_dot  # 导入模型可视化工具，用于生成模型结构图

# 加载预训练的InceptionNet模型（Inception v3）
# pretrained=True表示加载在ImageNet上预训练的权重
inception_model = models.inception_v3(pretrained=True)

# 打印模型结构，显示所有层和连接
print("InceptionNet模型结构:")
print(inception_model)

# 将模型移至GPU（如果可用）以加速计算，否则使用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception_model = inception_model.to(device)

# 使用torchsummary打印模型参数统计信息（层数、参数量等）
print("\nInceptionNet模型参数统计:")
torchsummary.summary(inception_model, (3, 299, 299))  # Inception v3需要299x299的输入图像，3表示RGB三通道

# 创建一个随机输入张量来可视化模型
# 设置模型为评估模式以避免批归一化层的错误
inception_model.eval()  # 将模型设置为评估模式，关闭dropout等训练特有操作
with torch.no_grad():  # 不计算梯度，节省内存
    dummy_input = torch.randn(1, 3, 299, 299).to(device)  # 创建一个随机输入张量，模拟一张299x299的RGB图像
    output = inception_model(dummy_input)  # 前向传播，获取模型输出

# 使用torchviz可视化模型结构并保存为图片
# 修复AttributeError: 'Tensor' object has no attribute 'logits'
# Inception v3在eval模式下直接返回tensor而不是包含logits属性的对象
model_graph = make_dot(output, params=dict(inception_model.named_parameters()))  # 创建计算图可视化对象
model_graph.render("inception_model", format="png")  # 渲染并保存为PNG图片




d:\Soft\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Soft\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\ArkhamCraft/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [01:15<00:00, 1.44MB/s] 


InceptionNet模型结构:
Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel

'inception_model.png'

In [ ]:
import json
token = {"username":"eviltrashcan","key":"5a0836e4ef86086fdadf624206bd1421"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)

In [ ]:
!cat /content/kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

In [ ]:
!kaggle competitions download -c cifar-10

In [ ]:
!unzip /content/competitions/cifar-10/cifar-10.zip

In [ ]:
%pip install py7zr
import py7zr
a =py7zr.SevenZipFile(r'./train.7z','r')
a.extractall(path=r'./competitions/cifar-10/')
a.close()

In [ ]:
!ls competitions/cifar-10/train|wc -l

In [5]:
# 加载CIFAR-10数据集
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

# 定义CIFAR-10数据集类
class CIFAR10Dataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None):
        self.img_dir = img_dir
        self.transform = transform

        # 读取标签文件，read_csv默认读取第一行作为列名
        self.labels_df = pd.read_csv(labels_file)
        self.img_names = self.labels_df.iloc[:, 0].values.astype(str)  # 第一列是图片名称，确保为字符串类型

        # 类别名称字典，使用字典可以提高查找速度
        self.class_names_dict = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3,
                                 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
        # 将文本标签转换为数字ID
        self.labels = [self.class_names_dict[label] for label in self.labels_df.iloc[:, 1].values]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx] + '.png') #图片路径
        image = Image.open(img_path) #打开图片
        label = self.labels[idx]

        if self.transform:
            image_tensor = self.transform(image)

        return image_tensor, label

# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4917, 0.4823, 0.4467), (0.2024, 0.1995, 0.2010))
])

# colab加载CIFAR-10数据集
# img_dir = r"competitions/cifar-10/train"
# labels_file = r"./trainLabels.csv"

img_dir = r"D:\BaiduNetdiskDownload\1.Python11期\深度学习代码\cifar-10\train"
labels_file = r"D:\BaiduNetdiskDownload\1.Python11期\深度学习代码\cifar-10\trainLabels.csv"
full_dataset = CIFAR10Dataset(img_dir=img_dir, labels_file=labels_file, transform=transform)

# 定义类别名称
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# 划分训练集和验证集
train_size = 45000
val_size = 5000
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(
    full_dataset,
    [train_size, val_size],
    generator=generator
)

# 查看数据集基本信息
print(f"完整数据集大小: {len(full_dataset)}")
print(f"训练集大小: {len(train_dataset)}")
print(f"验证集大小: {len(val_dataset)}")


完整数据集大小: 50000
训练集大小: 45000
验证集大小: 5000


In [6]:

# 创建数据加载器
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True #打乱数据集，每次迭代时，数据集的顺序都会被打乱
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False
)




In [7]:
# 自定义InceptionNet模型
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# 定义Inception模块
class InceptionModule(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        """
        Inception模块初始化
        
        参数:
            in_channels: 输入通道数
            ch1x1: 1x1卷积分支的输出通道数
            ch3x3red: 3x3卷积分支中1x1卷积的输出通道数(降维)
            ch3x3: 3x3卷积分支中3x3卷积的输出通道数
            ch5x5red: 5x5卷积分支中1x1卷积的输出通道数(降维)
            ch5x5: 5x5卷积分支中5x5卷积的输出通道数
            pool_proj: 池化分支中1x1卷积的输出通道数
        """
        super(InceptionModule, self).__init__()
        
        # 1x1卷积分支 - 直接进行特征提取和降维
        self.branch1 = nn.Conv2d(in_channels, ch1x1, kernel_size=1)
        
        # 1x1卷积 -> 3x3卷积分支 - 先降维再提取特征
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1),  # 降维
            nn.ReLU(inplace=True),  # 激活函数
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)  # 特征提取
        )
        
        # 1x1卷积 -> 5x5卷积分支 - 先降维再提取更大感受野的特征
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1),  # 降维
            nn.ReLU(inplace=True),  # 激活函数
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)  # 大感受野特征提取
        )
        
        # 3x3池化 -> 1x1卷积分支 - 先池化再降维
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),  # 池化提取主要特征
            nn.Conv2d(in_channels, pool_proj, kernel_size=1)  # 降维
        )
    
    def forward(self, x):
        """前向传播函数"""
        # 对每个分支进行计算并应用ReLU激活函数
        branch1 = F.relu(self.branch1(x))
        branch2 = F.relu(self.branch2(x))
        branch3 = F.relu(self.branch3(x))
        branch4 = F.relu(self.branch4(x))
        
        # 在通道维度上拼接所有分支的输出，形成多尺度特征表示
        return torch.cat([branch1, branch2, branch3, branch4], 1)

# 定义自定义的InceptionNet模型
class CustomInceptionNet(nn.Module):
    def __init__(self, num_classes=10):
        """
        自定义InceptionNet模型初始化
        
        参数:
            num_classes: 分类类别数，默认为10
        """
        super(CustomInceptionNet, self).__init__()
        
        # 初始卷积层 - 提取基础特征
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)  # 输入图像为3通道
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  # 降低分辨率
        
        # 卷积层 - 进一步提取特征
        self.conv2 = nn.Conv2d(64, 64, kernel_size=1)  # 1x1卷积进行通道整合
        self.conv3 = nn.Conv2d(64, 192, kernel_size=3, padding=1)  # 3x3卷积提取特征
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  # 再次降低分辨率
        
        # Inception模块 - 多尺度特征提取
        # 第一组Inception模块
        self.inception3a = InceptionModule(192, 64, 96, 128, 16, 32, 32)  # 输入192通道，输出256通道
        self.inception3b = InceptionModule(256, 128, 128, 192, 32, 96, 64)  # 输入256通道，输出480通道
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  # 降低分辨率
        
        # 第二组Inception模块
        self.inception4a = InceptionModule(480, 192, 96, 208, 16, 48, 64)  # 输入480通道，输出512通道
        self.inception4b = InceptionModule(512, 160, 112, 224, 24, 64, 64)  # 输入512通道，输出512通道
        
        # 全局平均池化 - 将特征图转换为特征向量
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # 自适应平均池化到1x1
        
        # 分类器
        self.dropout = nn.Dropout(0.4)  # 防止过拟合
        self.fc = nn.Linear(512, num_classes)  # 全连接层进行分类
    
    def forward(self, x):
        """前向传播函数"""
        # 初始层处理
        x = F.relu(self.conv1(x))  # 应用激活函数
        x = self.maxpool1(x)  # 池化
        
        # 卷积层处理
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.maxpool2(x)
        
        # Inception模块处理
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        x = self.inception4a(x)
        x = self.inception4b(x)
        
        # 全局平均池化
        x = self.avgpool(x)
        x = torch.flatten(x, 1)  # 展平特征图为一维向量
        
        # 分类器
        x = self.dropout(x)  # 应用dropout
        x = self.fc(x)  # 全连接层分类
        
        return x

# 创建自定义InceptionNet模型实例
model = CustomInceptionNet(num_classes=10)  # 创建10分类的模型实例

print(model)  # 打印模型结构


CustomInceptionNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception3a): InceptionModule(
    (branch1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (branch2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (branch3): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (branch4): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, padding=1

In [8]:
# 从train_loader获取第一个批次的数据
dataiter = iter(train_loader)
images, labels = next(dataiter)

# 查看批次数据的形状
print("批次图像形状:", images.shape)
print("批次标签形状:", labels.shape)


print('-'*100)
# 进行前向传播
with torch.no_grad():  # 不需要计算梯度
    outputs = model(images)


print(outputs.shape)

批次图像形状: torch.Size([64, 3, 32, 32])
批次标签形状: torch.Size([64])
----------------------------------------------------------------------------------------------------
torch.Size([64, 10])


In [9]:
# 计算模型的总参数量
# 统计需要求梯度的参数总量
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"需要求梯度的参数总量: {total_params}")

# 统计所有参数总量
all_params = sum(p.numel() for p in model.parameters())
print(f"模型总参数量: {all_params}")

# 查看每层参数量明细
print("\n各层参数量明细:")
for name, param in model.named_parameters():
    print(f"{name}: {param.numel()} 参数")

需要求梯度的参数总量: 1507314
模型总参数量: 1507314

各层参数量明细:
conv1.weight: 9408 参数
conv1.bias: 64 参数
conv2.weight: 4096 参数
conv2.bias: 64 参数
conv3.weight: 110592 参数
conv3.bias: 192 参数
inception3a.branch1.weight: 12288 参数
inception3a.branch1.bias: 64 参数
inception3a.branch2.0.weight: 18432 参数
inception3a.branch2.0.bias: 96 参数
inception3a.branch2.2.weight: 110592 参数
inception3a.branch2.2.bias: 128 参数
inception3a.branch3.0.weight: 3072 参数
inception3a.branch3.0.bias: 16 参数
inception3a.branch3.2.weight: 12800 参数
inception3a.branch3.2.bias: 32 参数
inception3a.branch4.1.weight: 6144 参数
inception3a.branch4.1.bias: 32 参数
inception3b.branch1.weight: 32768 参数
inception3b.branch1.bias: 128 参数
inception3b.branch2.0.weight: 32768 参数
inception3b.branch2.0.bias: 128 参数
inception3b.branch2.2.weight: 221184 参数
inception3b.branch2.2.bias: 192 参数
inception3b.branch3.0.weight: 8192 参数
inception3b.branch3.0.bias: 32 参数
inception3b.branch3.2.weight: 76800 参数
inception3b.branch3.2.bias: 96 参数
inception3b.branch4.1.weight: 163

In [10]:
# 定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()  # 交叉熵损失函数，适用于多分类问题，里边会做softmax，还有会把0-9标签转换成one-hot编码
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # SGD优化器，学习率为0.01，动量为0.9
print("损失函数:", loss_fn)

损失函数: CrossEntropyLoss()


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")
model = model.to(device) #将模型移动到GPU
early_stopping=EarlyStopping(patience=5, delta=0.001)
model_saver=ModelSaver(save_dir='model_weights', save_best_only=True)


model, history = train_classification_model(model, train_loader, val_loader, loss_fn, optimizer, device, num_epochs=50, early_stopping=early_stopping, model_saver=model_saver, tensorboard_logger=None)



使用设备: cuda:0


Training Progress:   0%|          | 0/35200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
history['train'][-100:-1]

In [ ]:
history['val'][-1000:-1]

In [ ]:
plot_learning_curves(history, sample_step=500)  #横坐标是 steps